In [1]:
import ase
import numpy as np
from numpy.linalg import norm, svd
from ase.visualize import view
import struct_statistics as stats
import matplotlib.pyplot as p
import soaputils as su
import soaplite
import genBasis
%matplotlib

Using matplotlib backend: TkAgg


In [2]:
%matplotlib
rCut = 10.0
NradBas = 5
Lmax = 5
myAlphas, myBetas = genBasis.getBasisFunc(rCut, NradBas)
n = 20

struct0 = ase.io.read("res_structs/dump_50A.cfg")
struct0s = struct0.copy()
struct0s.set_cell((15,15,15))
struct0s = su.limit_pos(struct0s)
print(struct0s.get_cell())
print(len(struct0s.get_chemical_symbols()))
#view(struct0s)

Using matplotlib backend: TkAgg
Cell([15.0, 15.0, 15.0], pbc=True)
197


In [3]:
# function to return vector with svd-norm for increasing offsets
np.random.seed(50)
def sample_vec(struct,cell,seed,method=2):
    struct = su.ran_sample(struct,cell,seed)
    n = 20
    s_norm = np.zeros(n)
    offset = np.linspace(0,9,n)
    for i in np.arange(n):
        struct_off = struct.copy()
        pos = struct_off.get_positions()
        shape = pos.shape
        #s = np.zeros(N)
        ran_pos = (np.random.random_sample(shape)-0.5)*2*offset[i]
        pos += ran_pos
        struct.set_positions(pos)
        struct.wrap(pbc=(1,1,1)) # atoms are wrapped inside unit cell to prevent atoms going outside of it
        mat = soaplite.get_periodic_soap_structure(struct_off, myAlphas, myBetas, rCut, NradBas, Lmax)
        s = svd(mat.transpose(), full_matrices=False, compute_uv=False)
        if method==0:
            s_norm[i] = norm(s, ord=1)
        elif method==1:
            s_norm[i] = norm(s)
        else:
            s_norm[i] = norm(s, ord=1)/norm(s)
    return s_norm

In [ ]:
# l1/l2 metric test
n = 20
np.random.seed(50)
Nsamples = 20
res1 = np.zeros((Nsamples,20))
offset = np.linspace(0,9,n)
for i in np.arange(Nsamples):
    print('step: %i' %i)
    vec = sample_vec(struct0, (15,15,15), 50+i)
    res1[i] = vec
np.save("res_data/offset_l1l2",res1)

In [ ]:
%matplotlib
res = np.load("res_data/offset_l1l2.npy")
p.plot(offset,res1.transpose())
p.xlabel('Random offset magnitude [$\AA$]')
p.ylabel('l1/l2 metric [a.u.]')

In [ ]:
var1 = np.std(res1, 0)
mean1 = np.mean(res1, 0)
offset = np.linspace(0,9,n)
p.errorbar(x=offset, y=mean1, yerr=var1)
p.xlabel('Random offset magnitude [$\AA$]')
p.ylabel('l1/l2 metric [a.u.]')

In [ ]:
# l1/l2 metric single species
structr = struct0.copy()
nums = structr.get_atomic_numbers()
nums = [1 for i in nums]
structr.set_atomic_numbers(nums)
#view(structr)

np.random.seed(50)
Nsamples = 20
res3 = np.zeros((Nsamples,20))
offset = np.linspace(0,9,n)
for i in np.arange(Nsamples):
    print('step: %i' %i)
    vec = sample_vec(structr, (15,15,15), 50+i)
    res3[i] = vec
np.save("res_data/offset_l1l2_single",res3)

In [ ]:
res = np.load("res_data/offset_l1l2_single.npy")
p.plot(offset,res.transpose())
p.xlabel('Random offset magnitude [$\AA$]')
p.ylabel('l1/l2 metric [a.u.]')

In [ ]:
var = np.std(res, 0)
mean = np.mean(res, 0)
offset = np.linspace(0,9,n)
p.errorbar(x=offset, y=mean, yerr=var)
p.xlabel('Random offset magnitude [$\AA$]')
p.ylabel('l1/l2 metric [a.u.]')

In [ ]:
# l1/l2 random starting positions
struct = su.rand_pos(struct0, seed=50)

np.random.seed(50)
Nsamples = 20
res4 = np.zeros((Nsamples,20))
offset = np.linspace(0,9,n)
for i in np.arange(Nsamples):
    print('step: %i' %i)
    vec = sample_vec(struct, (15,15,15), 50+i)
    res4[i] = vec
np.save("res_data/offset_l1l2_rand",res4)

In [ ]:
res = np.load("res_data/offset_l1l2_rand.npy")
p.plot(offset,res.transpose())
p.xlabel('Random offset magnitude [$\AA$]')
p.ylabel('l1/l2 metric [a.u.]')

In [ ]:
var = np.std(res, 0)
mean = np.mean(res, 0)
offset = np.linspace(0,9,n)
p.errorbar(x=offset, y=mean, yerr=var)
p.xlabel('Random offset magnitude [$\AA$]')
p.ylabel('l1/l2 metric [a.u.]')

In [ ]:
# l1 metric test
np.random.seed(50)
Nsamples = 20
res5 = np.zeros((Nsamples,20))
offset = np.linspace(0,9,n)
for i in np.arange(Nsamples):
    print('step: %i' %i)
    vec = sample_vec(struct0, (15,15,15), 50+i,method=0)
    res5[i] = vec
np.save("res_data/offset_l1",res5)

In [ ]:
res = np.load("res_data/offset_l1.npy")
p.plot(offset,res.transpose())
p.xlabel('Random offset magnitude [$\AA$]')
p.ylabel('l1 metric [a.u.]')

In [ ]:
var = np.std(res, 0)
mean = np.mean(res, 0)
offset = np.linspace(0,9,n)
p.errorbar(x=offset, y=mean, yerr=var)
p.xlabel('Random offset magnitude [$\AA$]')
p.ylabel('l1 metric [a.u.]')

In [6]:
# l2 metric test
np.random.seed(50)
Nsamples = 20
res6 = np.zeros((Nsamples,20))
offset = np.linspace(0,9,n)
for i in np.arange(Nsamples):
    print('step: %i' %i)
    vec = sample_vec(struct0, (15,15,15), 50+i,method=1)
    res6[i] = vec
np.save("res_data/offset_l2",res6)

step: 0
step: 1
step: 2
step: 3
step: 4
step: 5
step: 6
step: 7
step: 8
step: 9
step: 10
step: 11
step: 12
step: 13
step: 14
step: 15
step: 16
step: 17
step: 18
step: 19


In [9]:
res = np.load("res_data/offset_l2.npy")
p.plot(offset,res.transpose())
p.xlabel('Random offset magnitude [$\AA$]')
p.ylabel('l1 metric [a.u.]')

Text(0, 0.5, 'l1 metric [a.u.]')

In [10]:
var = np.std(res, 0)
mean = np.mean(res, 0)
offset = np.linspace(0,9,n)
p.errorbar(x=offset, y=mean, yerr=var)
p.xlabel('Random offset magnitude [$\AA$]')
p.ylabel('l2 metric [a.u.]')

Text(0, 0.5, 'l2 metric [a.u.]')

In [ ]:
fig, axs = p.subplots(1,2,figsize=[9,4],sharey=True)
axs[0].plot(offset,res6.transpose())
axs[0].set_xlabel('Random offset magnitude')
axs[0].set_ylabel('l2 metric')

axs[1].errorbar(x=offset, y=mean6, yerr=var6)
axs[0].set_xlabel('Random offset magnitude')
axs[0].set_ylabel('mean l2 metric')

In [ ]:
struct = su.gen_struct()